<h1>Capítulo 12</h1>

In [22]:
library(car);library(foreign);library(dynlm);library(lmtest);library(orcutt);library(sandwich);library(pdfetch)

Warning message:
"package 'pdfetch' was built under R version 3.6.3"


In [2]:
#exemplo 12.1: calculando a curva de philips e a curva de philips aumentada
data_phillips = read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\PHILLIPS.dta")
head(data_phillips)

Warning message in read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\PHILLIPS.dta"):
"cannot read factor labels from Stata 5 files"


,year,unem,inf,unem_1,inf_1,unem_2,inf_2,cunem,cinf,cunem_1,cinf_1
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1948,3.8,8.1,NA,NA,NA,NA,NA,NA,NA,NA
2,1949,5.9,-1.2,3.8,8.1,NA,NA,2.1000001,-9.3,NA,NA
3,1950,5.3,1.3,5.9,-1.2,3.8,8.1,-0.5999999,2.5,2.1000001,-9.3
4,1951,3.3,7.9,5.3,1.3,5.9,-1.2,-2.0000002,6.6,-0.5999999,2.5
5,1952,3.0,1.9,3.3,7.9,5.3,1.3,-0.3000000,-6.0,-2.0000002,6.6
6,1953,2.9,0.8,3.0,1.9,3.3,7.9,-0.0999999,-1.1,-0.3000000,-6.0


In [3]:
tsdata = ts(data_phillips, start=1948)

#curva de phillips normal
reg.s = dynlm(inf ~ unem, data=tsdata,end=1996)
residual.s = resid(reg.s)

coeftest(dynlm(residual.s ~ L(residual.s))) #erros e t statistic dos 
								 #residuos com eles laggados,
								 # que é forte evidencia de correlação


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   -0.11340    0.35940 -0.3155    0.7538    
L(residual.s)  0.57297    0.11613  4.9337 1.098e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [4]:
#curva de phillips aumentada

reg.ea = dynlm(d(inf) ~ unem, data=tsdata, end=1996)

residual.ea = resid(reg.ea) #pegar os residuos

coeftest(dynlm(residual.ea ~ L(residual.ea)))



t test of coefficients:

                Estimate Std. Error t value Pr(>|t|)
(Intercept)     0.194166   0.300384  0.6464   0.5213
L(residual.ea) -0.035593   0.123891 -0.2873   0.7752


In [5]:
##############
bgtest(reg.ea, type="F", order=1) # é pra automatizar, mas não sei como
##############


	Breusch-Godfrey test for serial correlation of order up to 1

data:  reg.ea
LM test = 0.057872, df1 = 1, df2 = 45, p-value = 0.811


In [8]:
#exemplo 12.3: autocorrelacao nos erros com AR(3)



barium = read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\BARIUM.dta")

tsdata = ts(barium, start=c(1978, 2), frequency=12)
reg = dynlm(log(chnimp) ~ log(chempi) + log(gas) + log(rtwex) +
				    befile6 + affile6 + afdec6, data=tsdata)


Warning message in read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\BARIUM.dta"):
"cannot read factor labels from Stata 5 files"


In [9]:
#teste manual

residual = resid(reg)
resreg = dynlm(residual ~ L(residual) + L(residual, 2) +
					L(residual, 3) + log(chempi) + 
					log(gas)+log(rtwex)+befile6+
					affile6+afdec6, data=tsdata)

linearHypothesis(resreg, 
			c("L(residual)", "L(residual, 2)", "L(residual, 3)"))



,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,121,43.39419,NA,NA,NA,NA
2,118,38.39367,3,5.000522,5.122907,0.002289803


In [10]:
#teste automatico usando o Lagrange multiplier (dá o resultado um pouco diferente do livro):

bgtest(reg, order=3, type="F")


	Breusch-Godfrey test for serial correlation of order up to 3

data:  reg
LM test = 5.1247, df1 = 3, df2 = 121, p-value = 0.002264


In [11]:
#exmplo do teste Durbin-watson, usado na msm ideia que o LM

library(foreign);library(dynlm);library(lmtest)

data_phillips = read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\PHILLIPS.dta")

tsdata = ts(data_phillips, start=1948)

reg.s = dynlm(inf ~ unem, data=tsdata, end=1996)

reg.ea = dynlm(d(inf) ~ unem, data=tsdata, end=1996)

Warning message in read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\PHILLIPS.dta"):
"cannot read factor labels from Stata 5 files"


In [12]:
dwtest(reg.s) #rejeitada


	Durbin-Watson test

data:  reg.s
DW = 0.8027, p-value = 7.552e-07
alternative hypothesis: true autocorrelation is greater than 0


In [13]:
dwtest(reg.ea) #aumentada rejeitada


	Durbin-Watson test

data:  reg.ea
DW = 1.7696, p-value = 0.1783
alternative hypothesis: true autocorrelation is greater than 0


In [15]:
#exemplo 12.4: Cochrane-Orcutt (FGLS estimation)



data_barium = read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\barium.dta")

tsdata = ts(data_barium, start=c(1978,2), frequency=12)

olsres = dynlm(log(chnimp) ~ log(chempi)+log(gas)+log(rtwex)+
			befile6+affile6+afdec6, data=tsdata) 
cochrane.orcutt(olsres)

Warning message in read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\barium.dta"):
"cannot read factor labels from Stata 5 files"


Cochrane-orcutt estimation for first order autocorrelation 
 
Call:
dynlm(formula = log(chnimp) ~ log(chempi) + log(gas) + log(rtwex) + 
    befile6 + affile6 + afdec6, data = tsdata)

 number of interaction: 8
 rho 0.293362

Durbin-Watson statistic 
(original):    1.45841 , p-value: 1.688e-04
(transformed): 2.06330 , p-value: 4.91e-01
 
 coefficients: 
(Intercept) log(chempi)    log(gas)  log(rtwex)     befile6     affile6 
 -37.322241    2.947434    1.054858    1.136918   -0.016372   -0.033082 
     afdec6 
  -0.577158 

In [17]:
#exemplo 12.7: serial correlation-robust inference with ols

#a heterocedasticidade e correlacao serial sao unbiased e consistent,
# mas o erro padrao sao prejudicados; para arrumar, temos

data_puertorico = read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\prminwge.dta")
head(data_puertorico)

Warning message in read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\prminwge.dta"):
"cannot read factor labels from Stata 5 files"


,year,avgmin,avgwage,kaitz,avgcov,covt,mfgwage,prdef,prepop,prepopf,...,lprunemp,lprgnp,lusgnp,lkaitz,lprun_1,lprepop,lprep_1,mincov,lmincov,lavgmin
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1950,0.198,0.398,0.155,0.201,0.29,0.43,0.859,0.470,0.470,...,2.734367,6.778443,7.093155,-1.864330,NA,-0.7550226,NA,0.09999498,-2.302635,-1.6194882
2,1951,0.209,0.410,0.164,0.207,0.29,0.45,0.881,0.449,0.449,...,2.772589,6.829794,7.191580,-1.807889,-1.870803,-0.8007324,-0.7550226,0.10551952,-2.248859,-1.5654210
3,1952,0.225,0.421,0.180,0.226,0.29,0.48,0.953,0.434,0.434,...,2.694627,6.923530,7.229839,-1.714798,-1.832582,-0.8347108,-0.8007324,0.12078384,-2.113753,-1.4916549
4,1953,0.311,0.480,0.229,0.231,0.29,0.50,0.970,0.428,0.428,...,2.674149,6.985919,7.269129,-1.474033,-1.910543,-0.8486321,-0.8347108,0.14966875,-1.899331,-1.1679624
5,1954,0.313,0.508,0.211,0.224,0.29,0.52,1.000,0.415,0.415,...,2.727853,7.007058,7.255733,-1.555897,-1.931022,-0.8794768,-0.8486321,0.13801575,-1.980387,-1.1615521
6,1955,0.369,0.547,0.231,0.236,0.29,0.57,1.003,0.419,0.419,...,2.580217,7.037467,7.309814,-1.465338,-1.877317,-0.8698844,-0.8794768,0.15920292,-1.837576,-0.9969587


In [18]:
tsdata = ts(data_puertorico, start=1950)

reg = dynlm(log(prepop) ~ log(mincov) + log(prgnp) + log(usgnp) + trend(tsdata), data=tsdata)

#resultados normais
coeftest(reg)


t test of coefficients:

                Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   -6.6634416  1.2578286 -5.2976 7.667e-06 ***
log(mincov)   -0.2122612  0.0401523 -5.2864 7.924e-06 ***
log(prgnp)     0.2852380  0.0804921  3.5437  0.001203 ** 
log(usgnp)     0.4860483  0.2219825  2.1896  0.035731 *  
trend(tsdata) -0.0266633  0.0046267 -5.7629 1.940e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [19]:
#resultados com a correção
coeftest(reg, vcovHAC)


t test of coefficients:

                Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   -6.6634416  1.6856885 -3.9529 0.0003845 ***
log(mincov)   -0.2122612  0.0460684 -4.6075 5.835e-05 ***
log(prgnp)     0.2852380  0.1034901  2.7562 0.0094497 ** 
log(usgnp)     0.4860483  0.3108940  1.5634 0.1275013    
trend(tsdata) -0.0266633  0.0054301 -4.9103 2.402e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [20]:
#exemplo 12.9 arch em stocks returns

nyse = read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\nyse.dta")

tsdata = ts(nyse)

reg = dynlm(return ~ L(return), data=tsdata)

residual.sq = resid(reg)^2

ARCHreg =  dynlm(residual.sq ~L(residual.sq))

coeftest(ARCHreg) #extremamente significativo

Warning message in read.dta("C:\\Users\\vinic\\Desktop\\R\\heiss\\data_wooldridge\\nyse.dta"):
"cannot read factor labels from Stata 5 files"



t test of coefficients:

               Estimate Std. Error t value  Pr(>|t|)    
(Intercept)    2.947433   0.440234  6.6951 4.485e-11 ***
L(residual.sq) 0.337062   0.035947  9.3767 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [25]:
#exemplo 12.7
dataset = zoo(pdfetch_YAHOO("GE", fields="adjclose", from="2000-01-01", to="2013-12-31"))

GE.ret = diff(log(dataset$GE))

In [28]:
#AR(1)
reg = dynlm(GE.ret ~L(GE.ret))

residual.sq = resid(reg)^2

ARCHreg = dynlm(residual.sq ~ L(residual.sq))

summary(ARCHreg)


Time series regression with "zoo" data:
Start = 2000-01-06, End = 2013-12-30

Call:
dynlm(formula = residual.sq ~ L(residual.sq))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.006851 -0.000326 -0.000283 -0.000086  0.031717 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    3.193e-04  2.279e-05   14.02   <2e-16 ***
L(residual.sq) 2.509e-01  1.633e-02   15.37   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.001287 on 3515 degrees of freedom
Multiple R-squared:  0.06294,	Adjusted R-squared:  0.06267 
F-statistic: 236.1 on 1 and 3515 DF,  p-value: < 2.2e-16
